# Set up the matrix

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from walker import *
from grid import *
import matplotlib.pyplot as plt
import scipy
import scipy.sparse as sp
import scipy.sparse.linalg as lg
from numba import jit
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from dask import delayed, compute
import matplotlib.path as mplpath

In [3]:
g = 4 # recursion depth
pts = 2 # points between each corner of the fractal
%time position_dict = koch_walker(depth=g, points_between=pts)
save_grid(position_dict, g, pts)

Wall time: 7.49 s


Load grid

In [4]:
# position_dict = load_grid(g, pts).item()

In [5]:
pos_list = np.array(list(position_dict))

In [6]:
max_idx = normalize(pos_list)
normalized_dict = 
max_idx

1278

In [59]:
%time ndict = {tuple(p):True for p in pos_list}

Wall time: 81.8 ms


solve eigenvalues

## point inside?
---
### matplotlib.path

In [39]:
path=mplpath.Path(pos_list)

In [40]:
points = np.random.randint(0, high =max_idx, size = (10000,2))

In [41]:
%time path.contains_points(points)

Wall time: 3.07 s


array([False, False, False, ...,  True,  True, False])

In [42]:
points

array([[1142, 1222],
       [1096, 1198],
       [1220,  112],
       ...,
       [ 395,  574],
       [1203,  736],
       [ 514,  395]])

### shapely

In [49]:
%time polygon = Polygon(pos_list)
%time contains = [Point(point).within(polygon) for point in points[0:200]]

#contains = [Polygon.contains(point) for point in testpoints]

Wall time: 1.99 ms
Wall time: 2.69 s


### Ray tracing

In [50]:
def ray_tracing(point, pos_dict, max_idx):
    if pos_dict.get(tuple(point), False):
        return True
    sign = 1
    x0, y0 = point
    if x0 < max_idx//2:
        if y0 < x0:
            sign = -1
            idx = 1# direction: substract from y
        else:
            sign = -1
            idx = 0
            # direction substract from x
    else:
        if y0 > x0:
            idx=1
            # add to y
        else:
            idx = 0
            # add to x
    p = point
    inside = False
    
    while p[idx] > 0 & p[idx] < max_idx:
        temp = p[idx] + sign
        p[idx] = temp
        if pos_dict.get(tuple(p), default = False):
            inside = not inside
    return inside

In [74]:
fast_ray_tracing = jit(ray_tracing) # hvorfor er den så treg?

In [68]:
%time contains_ray = np.array([ray_tracing(point, ndict, max_idx) for point in pos_list[0:10000]])
#%time [fast_ray_tracing(point, position_dict, max_idx) for point in pos_list[0:10]]

Wall time: 20 ms


In [76]:
%time contains_ray = np.array([fast_ray_tracing(point, ndict, max_idx) for point in pos_list[0:10000]])

Wall time: 1.43 s


In [78]:
%%time
res = []
for point in pos_list[0:100]:
    a = delayed(ray_tracing)(point, position_dict, max_idx)
    res.append(a)
results =compute(res)

Wall time: 3min 3s


In [ ]:
res[0].visualize()